##Neural Network SMS Text Classifier

Create a machine learning model that will classify SMS messages as either "ham" or "spam". A "ham" message is a normal message sent by a friend. A "spam" message is an advertisement or a message sent by a company.

In [ ]:
# import libraries
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

2.19.0


In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

--2025-09-30 18:10:13--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.2.33, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv’

train-data.tsv      100%[===================>] 349.84K  --.-KB/s    in 0.02s   

2025-09-30 18:10:13 (13.9 MB/s) - ‘train-data.tsv’ saved [358233/358233]

--2025-09-30 18:10:13--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.2.33, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [text/tab-separated-values]
Saving to: ‘valid-data.tsv’

valid-data.tsv      100%[==========

In [ ]:
train_df = pd.read_csv(train_file_path, sep='\t', header=None, names=['label', 'message'])
test_df = pd.read_csv(test_file_path, sep='\t', header=None, names=['label', 'message'])

In [ ]:
train_labels = train_df['label'].map({'ham': 0, 'spam': 1}).values
test_labels = test_df['label'].map({'ham': 0, 'spam': 1}).values

In [ ]:
train_texts = train_df['message'].values
test_texts = test_df['message'].values

Vectorization

In [ ]:
from tensorflow.keras.layers import TextVectorization
max_features = 10000
sequence_length = 250

vectorizer = TextVectorization(
    max_tokens=max_features,
    output_mode="int",
    output_sequence_length=sequence_length
)
vectorizer.adapt(train_texts)

Build the Model

In [ ]:
from tensorflow.keras import layers
model = keras.Sequential([
    vectorizer,
    layers.Embedding(max_features + 1, 16),
    layers.GlobalAveragePooling1D(),
    layers.Dense(16, activation="relu"),
    layers.Dense(1, activation="sigmoid")
])

model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

Train the Model

In [ ]:
history = model.fit(
    train_texts, train_labels,
    epochs=30,
    validation_data=(test_texts, test_labels),
    verbose=2
)

Epoch 1/30
131/131 - 1s - 9ms/step - accuracy: 0.9890 - loss: 0.0394 - val_accuracy: 0.9734 - val_loss: 0.0974
Epoch 2/30
131/131 - 1s - 9ms/step - accuracy: 0.9873 - loss: 0.0404 - val_accuracy: 0.9792 - val_loss: 0.0616
Epoch 3/30
131/131 - 1s - 5ms/step - accuracy: 0.9909 - loss: 0.0349 - val_accuracy: 0.9698 - val_loss: 0.0933
Epoch 4/30
131/131 - 1s - 5ms/step - accuracy: 0.9904 - loss: 0.0373 - val_accuracy: 0.9784 - val_loss: 0.0604
Epoch 5/30
131/131 - 1s - 5ms/step - accuracy: 0.9914 - loss: 0.0331 - val_accuracy: 0.9770 - val_loss: 0.0754
Epoch 6/30
131/131 - 1s - 5ms/step - accuracy: 0.9914 - loss: 0.0327 - val_accuracy: 0.9777 - val_loss: 0.0641
Epoch 7/30
131/131 - 1s - 5ms/step - accuracy: 0.9914 - loss: 0.0325 - val_accuracy: 0.9806 - val_loss: 0.0577
Epoch 8/30
131/131 - 1s - 6ms/step - accuracy: 0.9911 - loss: 0.0310 - val_accuracy: 0.9770 - val_loss: 0.0701
Epoch 9/30
131/131 - 1s - 5ms/step - accuracy: 0.9914 - loss: 0.0286 - val_accuracy: 0.9770 - val_loss: 0.0722
E

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  input_text = tf.constant([pred_text])
  prob = model.predict(input_text)[0][0]
  label = "spam" if prob > 0.5 else "ham"
  return [float(prob), label]

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
[0.021219773218035698, 'ham']


Test the Model

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
You passed the challenge. Great job!
